In [1]:
import requests
import pandas as pd
import json
import helper

pd.set_option('display.max_rows', None)

weiDivisible = 1000000000000000000.0
tokenContract = "0xd8a4b3f4afd16eed638cbbd19d5801a5735c0930" #dejitaru paru
url = f"https://api.ethplorer.io/getTopTokenHolders/{tokenContract}?apiKey=freekey&limit=50"

response = requests.get(url)
response = response.json()
response = response['holders']
#data = json.loads(response)

df = pd.DataFrame.from_dict(response)

In [2]:
addresses = df['address']
isContract = []
for address in addresses:
    if(helper.isContract(address)):
        isContract.append(True)
    else: 
        isContract.append(False)
    
df["isContract"] = isContract
df2 = df[df["isContract"] == False]
df = df[df["isContract"] == True]
df.to_csv("result_contracts.csv")

In [3]:
addresses = df2["address"]
ethBalances = []
tokenBalances = []
for address in addresses:
    url = f"https://api.ethplorer.io/getAddressInfo/{address}?apiKey=EK-jCkqf-qQHnUdG-uyNuL"

    response = requests.get(url)
    response = response.json()

    ethBalance = response['ETH']['balance'] * response['ETH']['price']['rate']
    tokenBalance = helper.getTokenBalance(address)

    ethBalances.append(ethBalance)
    tokenBalances.append(tokenBalance)

In [4]:
df2['EthBalance'] = ethBalances
df2['TokenBalance'] = tokenBalances
df2["EthBalance"] = pd.to_numeric(df2["EthBalance"])
df2["TokenBalance"] = pd.to_numeric(df2["TokenBalance"])
df2['TotalBalance'] = df2["EthBalance"] + df2["TokenBalance"]

In [5]:
#   RESEARCH FINDINGS
# Fresh: Total Transaction <= 10 
# Good:  Daily Transaction <= 5 (Inactive wallets also 'Good')
# Aight: Daily Transaction 5 < x <= 10
# Jett:   Daily Transaction > 10

wallets = df2["address"]
walletLabel = []
goodWalletsToAnalyzeLater = []

for wallet in wallets:
    url=f"https://api.etherscan.io/api?module=account&action=tokentx&address={wallet}&page=1&offset=100&startblock=0&endblock=99999999&sort=desc&apikey=7YRRHUFSDSM5BXCIM8FY3NPJ192WYKRC1G"

    resp=requests.get(url)
    resp=resp.json()
    resp=resp['result']

    if len(resp) < 10: 
        walletLabel.append("Fresh")
        continue

    yesterdayTime = int(resp[0]['timeStamp']) - 86400
    walletTransfers = []
    for transfer in resp:
        if int(transfer['timeStamp']) >= yesterdayTime: walletTransfers.append(transfer['timeStamp'])

    walletTransfers = [*set(walletTransfers)]
    dailyTransfer = len(walletTransfers)

    if dailyTransfer <= 5: 
        walletLabel.append("Good")
        goodWalletsToAnalyzeLater.append(wallet)
    elif dailyTransfer > 5 and dailyTransfer <= 10: walletLabel.append("Aight")
    if dailyTransfer > 10: walletLabel.append("Jett")

with open("testDEJITARUPARUGoodWallets.csv", "w") as f:
    for wallet in goodWalletsToAnalyzeLater:
        f.write(wallet + "\n")

df2['Label'] = walletLabel

In [6]:
df2.to_csv("result.csv")